In [ ]:
"""
    algorithm: Prioritized Replay DDQN
        The experience will be sampled according to the TD-error.
        The greater the difference of TD-error(s,a), the higher the priority of (s,a) sampled.
        Certainly， the Loss function also include the weight of the priority.

        More details you can learn from the paper: https://arxiv.org/pdf/1511.05952
        key points:
            1. The method of sampled experience is imporved.
        
    environment: CartPole-v0
    state:
        1.Cart Position:[-4.8,4.8],  2.Cart Velocity[-Inf,Inf],  3.Pole Angle[-24 deg, 24 deg]
        4.Pole Velocity [-Inf,Inf]
    action:
        0: left    
        1: right
    reward: 1 for every step    
        
    prerequisites:  tensorflow 2.2(tensorflow >= 2.0)
    notice：
    
    author: Xinchen Han
    date: 2020/7/30

"""

In [ ]:
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import gym
import random

In [ ]:
"""Environment"""
env = gym.make('CartPole-v0')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

"""Random seed"""
env.seed(6)
np.random.seed(6)
random.seed(6)
tf.random.set_seed(6)

"""Set hyperparameters"""
alpha = 0.9
gamma = 0.9
max_episodes = 500

render = True

In [ ]:
class SumTree(object):
    """
    This SumTree code is from:
    https://github.com/ljpzzz/machinelearning/blob/master/reinforcement-learning/ddqn_prioritised_replay.py
    Story data with its priority in the tree.
    """

    data_pointer = 0

    def __init__(self, capacity):
        self.capacity = capacity  # for all priority values
        self.tree = np.zeros(2 * capacity - 1)
        # [--------------Parent nodes-------------][-------leaves to recode priority-------]
        #             size: capacity - 1                       size: capacity
        self.data = np.zeros(capacity, dtype=object)  # for all agent info.
        # [--------------data frame-------------]
        #             size: capacity

    def add(self, p, data):
        tree_idx = self.data_pointer + self.capacity - 1
        self.data[self.data_pointer] = data  # update data_frame
        self.update(tree_idx, p)  # update tree_frame

        self.data_pointer += 1
        if self.data_pointer >= self.capacity:  # replace when exceed the capacity
            self.data_pointer = 0

    def update(self, tree_idx, p):
        change = p - self.tree[tree_idx]
        self.tree[tree_idx] = p
        # then propagate the change through tree
        while tree_idx != 0:    # this method is faster than the recursive loop in the reference code
            tree_idx = (tree_idx - 1) // 2
            self.tree[tree_idx] += change

    def get_leaf(self, v):
        """
        Tree structure and array storage:
        Tree index:
             0         -> storing priority sum
            / \
          1     2
         / \   / \
        3   4 5   6    -> storing priority for transitions
        Array type for storing:
        [0,1,2,3,4,5,6]
        """
        parent_idx = 0
        while True:     # the while loop is faster than the method in the reference code
            cl_idx = 2 * parent_idx + 1         # this leaf's left and right kids
            cr_idx = cl_idx + 1
            if cl_idx >= len(self.tree):        # reach bottom, end search
                leaf_idx = parent_idx
                break
            else:       # downward search, always search for a higher priority node
                if v <= self.tree[cl_idx]:
                    parent_idx = cl_idx
                else:
                    v -= self.tree[cl_idx]
                    parent_idx = cr_idx

        data_idx = leaf_idx - self.capacity + 1
        return leaf_idx, self.tree[leaf_idx], self.data[data_idx]

    @property
    def total_p(self):
        return self.tree[0]  # the root

In [ ]:
class Prioritized_Replay_DQN(object):

    def __init__(self):
        self.step = 0
        self.sampling_nums = 0
        self.batch_size = 64
        self.update_freq = 100
        self.experience_nums = 0
        self.replay_size = 1000
        self.model = self.create_model()
        self.target_model = self.create_model()
        self.optimizer = keras.optimizers.Adam(1e-3)

        self.tree = SumTree(self.replay_size)
        self.epsilon = 0.01  # small amount to avoid zero priority
        self.alpha = 0.6  # [0~1] convert the importance of TD error to priority
        self.beta = 0.4  # importance-sampling, from initial value increasing to 1
        self.beta_increment_per_sampling = 0.001
        self.abs_err_upper = 1.  # clipped abs error

    def create_model(self):
        input = keras.layers.Input(shape=state_dim)
        hidden1 = keras.layers.Dense(64, activation='relu')(input)
        hidden2 = keras.layers.Dense(16, activation='tanh')(hidden1)
        output = keras.layers.Dense(action_dim)(hidden2)
        model = keras.models.Model(inputs=[input], outputs=[output])
        return model

    def choose_action(self, state, epsilon=0.1):
        if np.random.uniform() > epsilon - self.step * 0.0001:
            return np.argmax(self.model.predict(tf.convert_to_tensor([state], dtype=tf.float32))[0])
        else:
            return random.sample(list(np.arange(0, action_dim)), 1)[0]

    def perceive(self, state, action, state_, reward, done):
        # max_p = np.max(self.tree.tree[-self.tree.capacity: ])
        # if max_p == 0:
        #     max_p = self.abs_err_upper
        max_p = np.mean(np.abs(reward + gamma * self.target_model(np.array(state_).reshape(1,state_dim)) \
                       - self.model(np.array(state).reshape(1, state_dim))))
        self.tree.add(max_p, np.hstack((state, action, state_, reward, done)))   # set the max p for new p
        self.experience_nums += 1
        # print(max_p)
        # print(self.tree.total_p)

    def sample(self, size):
        b_idx, b_memory, ISWeights = np.empty((size,), dtype=np.int32), \
                                     np.empty((size, self.tree.data[0].size)), \
                                     np.empty((size, 1))
        pri_seg = self.tree.total_p / size       # priority segment
        self.beta = np.min([1., self.beta + self.beta_increment_per_sampling])  # max = 1

        min_prob = np.min(self.tree.tree[-self.tree.capacity:]) / self.tree.total_p     # for later calculate ISweight
        if min_prob == 0:
            min_prob = 0.00001
        for i in range(size):
            v = np.random.uniform(pri_seg * i, pri_seg * (i + 1))
            idx, p, data = self.tree.get_leaf(v)
            prob = p / self.tree.total_p
            ISWeights[i, 0] = np.power(prob/min_prob, -self.beta)
            b_idx[i], b_memory[i, :] = idx, data
        return b_idx, b_memory, ISWeights

    def model_train(self):
        self.step += 1
        if self.step % self.update_freq == 0:
            self.target_model.set_weights(self.model.get_weights())

        tree_idx, replay_batch, ISWeights = self.sample(self.batch_size)
        state_batch = replay_batch[:, 0:state_dim]
        action_batch = replay_batch[:, state_dim:state_dim + 1]
        next_state_batch = replay_batch[:, state_dim + 1:state_dim + 1 +state_dim]
        reward_batch = replay_batch[:, state_dim + 1 + state_dim:state_dim + 1 + state_dim + 1]
        done_batch = replay_batch[:, -1]


        with tf.GradientTape() as tape:
            tape.watch(self.model.variables)
            Q = self.model(tf.convert_to_tensor(state_batch, dtype=tf.float32))
            Q_next = self.target_model(tf.convert_to_tensor(next_state_batch, dtype=tf.float32))

            max_action = tf.argmax(Q, axis=1)  # Nature_DQN is tf.argmax(Q_next, axis=1)
            Q = tf.reduce_sum(tf.one_hot(action_batch, action_dim) * Q, axis=1)
            Q_next = tf.reduce_sum(tf.one_hot(max_action, action_dim) * Q_next, axis=1)

            target_value = np.array((1 - done_batch) * gamma * Q_next).reshape(64,1) + reward_batch
            loss = tf.reduce_mean(ISWeights * tf.square(Q - target_value) * 0.5)

        grads = tape.gradient(loss, self.model.variables)
        self.optimizer.apply_gradients(zip(grads, self.model.variables))


In [ ]:
if __name__ == "__main__":
    agent = Prioritized_Replay_DQN()
    score_list = []
    for episode in range(max_episodes):
        state = env.reset()
        score = 0
        while True:
            action = agent.choose_action(state)
            if render:
                env.render()
            state_, reward, done, _ = env.step(action)
            agent.perceive(state, action, state_, reward, done)
            if agent.experience_nums >= agent.replay_size:
                agent.model_train()
            state = state_
            score += reward
            if done:
                score_list.append(score)
                print('episode:', episode, 'score:', score, 'max_score:', np.max(score_list))
                if agent.experience_nums >= agent.replay_size:
                    print("   Training   ....")
                break
        if np.mean(score_list[-10:]) > 180:
            break

    env.close()
    plt.plot(score_list, color='orange')
    plt.show()

